<a href="https://colab.research.google.com/github/inuin09/Natural-language-Processing/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글 드라이브와 연결
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1==0.7.4
!pip install rhinoMorph

In [10]:
#파일이 있는 곳으로 경로를 변경한다.
%cd/content/gdrive/My\ Drive/pytest/

/content/gdrive/My Drive/pytest


In [11]:
#형태 분석된 데이터 로딩
#데이터 로딩
def read_data(filename, encoding='cp949'):  #읽기 함수 정의 , cp949 ??
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]  
    #txt 파일의 헤더(id document label)는 제외하기
    #문장 분리-> 각 문장에서 탭 부분 분리 -> 리스트 생성 ->전체로는 중첩 리스트
    data=data[1:]
  return data

def write_data(data, filename, encoding='cp949'):  #쓰기 함수도 정의 
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

data = read_data('ratings_morphed.txt', encoding='cp949')  #전체 파일은 ratings.txt
print(len(data))
print(len(data[0]))
print(data[0])

491
3
['4655635', '폴리스스토리 시리즈 뉴 없다 최고', '1']


In [12]:
#훈련 데이터와 테스트데이터 분리(자동)
data_text = [line[1] for line in data]  #데이터 본문
data_senti = [line[2] for line in data]  #데이터 긍부정 부분

from sklearn.model_selection import train_test_split #본문과 라벨을 각각 분리
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)

#Counter 클래스를 이용해 각 분류가 훈련데이터와 테스트데이터에 같은 비율로 들어갔는지 확인해본다.
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq : ', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq : ',test_data_senti_freq)



train_data_senti_freq :  Counter({'0': 184, '1': 184})
test_data_senti_freq :  Counter({'0': 62, '1': 61})


In [14]:
#훈련 데이터와 테스트데이터 분리(수동)
import random
random.shuffle(data)  #랜덤하게 섞는다

data_70 = int(len(data)*0.7)  #전체 데이터 크기의 70% 숫자를 찾는다
train_data = data[:data_70] #앞에서 70% 부분을 잘라 훈련데이터로
test_data = data[data_70:]  #그 뒷부분을 테스트 데이터로

print('train_data length :', len(train_data)) 
print('test_data length :', len(test_data))

#훈련 데이터 요소 분리
train_data_text = [line[1] for line in train_data]  #훈련데이터 본문
train_data_senti = [line[2] for line in train_data] #훈련 데이터 긍부정 부분

#테스트데이터 요소 분리
test_data_text = [line[1] for line in test_data]
test_data_senti = [line[2] for line in test_data]  

train_data length : 343
test_data length : 148


In [15]:
#행렬 형태로 변환
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_data_text) #최소 문서 빈도 5이상의 단어만 대상
X_train = vect.transform(train_data_text) #행렬 생성
print('X_train : \n', repr(X_train))

X_train : 
 <343x72 sparse matrix of type '<class 'numpy.int64'>'
	with 943 stored elements in Compressed Sparse Row format>


In [16]:
feature_names = vect.get_feature_names()
print('특성 개수 : ',len(feature_names))
print('처음 20개의 특성 : ',feature_names[:20])

특성 개수 :  72
처음 20개의 특성 :  ['10점', 'ㅋㅋ', 'ㅎㅎ', 'ㅠㅠ', 'ㅡㅡ', '감독', '감동', '같다', '그런', '그렇다', '나오다', '남다', '남자', '내내', '내용', '너무', '다시', '대박', '대하다', '되다']


In [17]:
#머신러닝 알고리즘 적용
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti) #리스트 형태를 종속변수가 될 수 있는 1차원 배열(시리즈)로 만든다

lr=LogisticRegression(solver='liblinear') #모델 생성
lr.fit(X_train, y_train) #모델 훈련

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
#테스트 데이터 입력
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)

print('테스트 데이터 점수 : ',lr.score(X_test, y_test))

테스트 데이터 점수 :  0.7094594594594594


In [20]:
#1개 데이터 예측
#형태소 분석기 시작
import rhinoMorph
rn = rhinoMorph.startRhino()

new_input = '오늘은 정말 재미있는 하루구나!'
inputdata=[]
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG','NNP','VV','VA','XR','IC','MM','MAG','MAJ'])
morphed_input = ' '.join(morphed_input)

inputdata.append(morphed_input)
print('input data :',inputdata)

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
input data : ['오늘 정말 재미있 하루']


In [21]:
X_input = vect.transform(inputdata)
result = lr.predict(X_input)

if result == '0':
  print('부정적')
else:
  print('긍정적')

부정적
